### Welcome to Module 5 Homework

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/02 07:24:07 WARN Utils: Your hostname, amr-VirtualBox resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface enp0s3)
24/03/02 07:24:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 07:24:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd

## Answer of question 1

In [3]:
pyspark.__version__

'3.5.0'

## Answer of question 2

In [4]:
df_fhv_pd = pd.read_csv('/home/amr/NYC_Taxi_Data/fhv_2019/fhv_tripdata_2019_10.csv', nrows=1000)

In [5]:
spark.createDataFrame(df_fhv_pd).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [6]:
from pyspark.sql import types

In [7]:
df_fhv_schema = types.StructType([
    types.StructField("dispatching_base_num", types.StringType(), True), 
    types.StructField("pickup_datetime", types.TimestampType(), True), 
    types.StructField("dropOff_datetime", types.TimestampType(), True), 
    types.StructField("PUlocationID", types.IntegerType(), True), 
    types.StructField("DOlocationID", types.IntegerType(), True), 
    types.StructField("SR_Flag", types.IntegerType(), True), 
    types.StructField("Affiliated_base_number", types.IntegerType(), True)
])

In [9]:
df_fhv = spark.read \
    .option("header", "true") \
    .schema(df_fhv_schema) \
    .csv('/home/amr/NYC_Taxi_Data/fhv_2019/fhv_tripdata_2019_10.csv')

df_fhv \
.repartition(6) \
.write.parquet('/home/amr/NYC_Taxi_Data/Module5/fhv_2029_10.parquet', mode='overwrite')

## Answer of question 3

In [10]:
df_fhv_pq = spark.read.parquet('/home/amr/NYC_Taxi_Data/Module5/*/*')

In [11]:
df_fhv_pq.registerTempTable('fhv_2019_10')

/home/amr/spark/spark-3.5.0-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [12]:
df_fhv_pq.head(10)

[Row(dispatching_base_num='B02293', pickup_datetime=datetime.datetime(2019, 10, 2, 9, 59, 29), dropOff_datetime=datetime.datetime(2019, 10, 2, 10, 34, 6), PUlocationID=22, DOlocationID=181, SR_Flag=None, Affiliated_base_number=None),
 Row(dispatching_base_num='B01145', pickup_datetime=datetime.datetime(2019, 10, 10, 22, 22, 2), dropOff_datetime=datetime.datetime(2019, 10, 10, 22, 25, 23), PUlocationID=264, DOlocationID=136, SR_Flag=None, Affiliated_base_number=None),
 Row(dispatching_base_num='B00477', pickup_datetime=datetime.datetime(2019, 10, 8, 1, 55, 43), dropOff_datetime=datetime.datetime(2019, 10, 8, 2, 7, 51), PUlocationID=79, DOlocationID=258, SR_Flag=None, Affiliated_base_number=None),
 Row(dispatching_base_num='B01437', pickup_datetime=datetime.datetime(2019, 10, 9, 12, 49, 40), dropOff_datetime=datetime.datetime(2019, 10, 9, 13, 0, 58), PUlocationID=264, DOlocationID=130, SR_Flag=None, Affiliated_base_number=None),
 Row(dispatching_base_num='B00256', pickup_datetime=datetim

In [13]:
Que3 = spark.sql("""
SELECT 
    date_trunc('day', pickup_datetime) AS PickUp_Day, 
    count(*) as no_of_trips   
FROM
    fhv_2019_10
WHERE
    date_trunc('day', pickup_datetime) = '2019-10-15'
GROUP BY
    1
""").show()

+-------------------+-----------+
|         PickUp_Day|no_of_trips|
+-------------------+-----------+
|2019-10-15 00:00:00|      62610|
+-------------------+-----------+



## Answer of question 4

In [14]:
Que3 = spark.sql("""
SELECT MAX(calculated_trip_length) AS longest_trip_hours
FROM (SELECT 
    pickup_datetime, 
    DATEDIFF(HOUR, pickup_datetime, dropOff_datetime) AS calculated_trip_length
FROM
    fhv_2019_10
order by
    date_trunc('hour', pickup_datetime)) 
AS trip_durations;
""").show()

+------------------+
|longest_trip_hours|
+------------------+
|            631152|
+------------------+



## Answer for question 6

In [15]:
df_zones = spark.read.parquet('zones/')

In [16]:
df_result = df_fhv_pq.join(df_zones, df_fhv_pq.PUlocationID== df_zones.LocationID, how='leftouter')
df_result.registerTempTable('fhv_2019_10_zones')

In [17]:
Que3 = spark.sql("""
SELECT Zone AS least_frequent_zone
FROM fhv_2019_10_zones
GROUP BY Zone
ORDER BY COUNT(*) ASC
LIMIT 1;
""").show()

+-------------------+
|least_frequent_zone|
+-------------------+
|        Jamaica Bay|
+-------------------+

